In [1]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 讀取資料
# 請確保 csv 檔案跟程式碼在同一個資料夾，或是修改路徑
df = pd.read_csv('tcc_ceds_music.csv')

# 取得數值型欄位供選單使用
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
# 移除無意義的 index 欄位
if 'Unnamed: 0' in numeric_cols:
    numeric_cols.remove('Unnamed: 0')

# 2. 定義繪圖函數

def show_data_head():
    """回傳資料的前 5 筆"""
    return df.head()

def plot_distribution(column_name):
    """繪製單一變數的分佈圖"""
    fig = plt.figure(figsize=(10, 6))
    sns.histplot(df[column_name], kde=True, bins=30, color='skyblue')
    plt.title(f'Distribution of {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.tight_layout()
    return fig

def plot_scatter(x_col, y_col, color_by_genre):
    """繪製兩個變數的散佈圖"""
    fig = plt.figure(figsize=(10, 6))
    if color_by_genre:
        sns.scatterplot(data=df, x=x_col, y=y_col, hue='genre', alpha=0.6)
    else:
        sns.scatterplot(data=df, x=x_col, y=y_col, alpha=0.6)

    plt.title(f'{x_col} vs {y_col}')
    plt.tight_layout()
    return fig

def plot_correlation():
    """繪製相關性熱圖"""
    fig = plt.figure(figsize=(12, 10))
    # 只計算數值欄位的相關性
    corr = df[numeric_cols].corr()
    sns.heatmap(corr, cmap='coolwarm', linewidths=0.5)
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    return fig

# 3. 建立 Gradio 介面 (使用 Blocks 排版)

with gr.Blocks(title="🎵 Music Data EDA Dashboard") as demo:
    gr.Markdown("# 🎵 音樂數據探索儀表板 (EDA Dashboard)")
    gr.Markdown("使用 Gradio 快速探索音樂特徵數據集！")

    with gr.Tabs():

        # 分頁 1: 數據總覽
        with gr.TabItem("數據總覽 (Overview)"):
            gr.Markdown("### 前 5 筆資料預覽")
            dataframe_output = gr.Dataframe()
            # 按鈕觸發
            show_btn = gr.Button("顯示數據")
            show_btn.click(show_data_head, outputs=dataframe_output)

        # 分頁 2: 單變量分析
        with gr.TabItem("單變量分佈 (Distribution)"):
            gr.Markdown("### 選擇一個特徵來查看其分佈")
            with gr.Row():
                col_input = gr.Dropdown(choices=numeric_cols, value='danceability', label="選擇欄位")
            plot_output_dist = gr.Plot()
            # 當選單改變時，自動更新圖表
            col_input.change(plot_distribution, inputs=col_input, outputs=plot_output_dist)

        # 分頁 3: 雙變量分析
        with gr.TabItem("變數關係 (Relation)"):
            gr.Markdown("### 探索兩個特徵之間的關係")
            with gr.Row():
                x_input = gr.Dropdown(choices=numeric_cols, value='loudness', label="X 軸特徵")
                y_input = gr.Dropdown(choices=numeric_cols, value='energy', label="Y 軸特徵")
                genre_check = gr.Checkbox(value=True, label="依曲風 (Genre) 著色")

            plot_output_scatter = gr.Plot()

            # 定義更新函數
            inputs = [x_input, y_input, genre_check]
            for input_component in inputs:
                input_component.change(plot_scatter, inputs=inputs, outputs=plot_output_scatter)

        # 分頁 4: 相關性分析
        with gr.TabItem("相關性熱圖 (Heatmap)"):
             gr.Markdown("### 特徵相關性矩陣")
             heatmap_btn = gr.Button("產生熱圖")
             heatmap_output = gr.Plot()
             heatmap_btn.click(plot_correlation, outputs=heatmap_output)

# 4. 啟動應用程式
if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63e6591e77004fe265.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
